# CPI inflation graphic
Date: *March 16, 2022*

A quick tutorial on how I prepared a graphic on CPI inflation for CBC News. We import Pandas, then we import the data.

In [1]:
import pandas as pd

raw = pd.read_csv("../raw/RAW 2022 STATSCAN CPI INFLATION.csv")

First we have to filter the data to show only what we want to see: all basket items, and only for Canada (not provinces or territories). We also filter for data from 2000 onwards.

In [91]:
filtered = raw[(raw["Products and product groups"] == "All-items")
               & (raw["GEO"] == "Canada")
               & (raw["REF_DATE"] >= "2000-01")
               ]

filtered = (filtered
            .pivot(index="REF_DATE", values="VALUE", columns="Products and product groups")
            .reset_index()
            )

Now we have to work some magic on the data. We'll start by calculating the % change vs. the same month in the previous year. Then, we get rid of empty values (really, just the first 12 values because they can't compare to anything!).

In [103]:
filtered["change"] = (filtered["All-items"].pct_change(12) * 100)
filtered = filtered.dropna(subset=["change"])
filtered["REF_DATE"] = pd.to_datetime(filtered["REF_DATE"])

Using this set, we'll do two version and join them: one for monthly changes over the previous year, and the other for only February changes, with interpolations in between (so we get a nice smooth line).

In [108]:
monthly = filtered[["REF_DATE", "change"]].set_index("REF_DATE")

annual = filtered[["REF_DATE", "change"]].set_index("REF_DATE")
annual = annual[annual.index.month == 2]

both = (monthly
        .merge(annual, how="left", on="REF_DATE")
        .interpolate(method="quadratic")
        .dropna()
        )

Now we take it over to Datawrapper, and here is the result: [https://www.datawrapper.de/_/LgM77/](https://www.datawrapper.de/_/LgM77/).